### Base de Datos Vectorial Pinecone



In [ ]:
## Crea tu índice y clave API desde aquí https://www.pinecone.io/
# Variable para almacenar tu clave API de Pinecone
# IMPORTANTE: Obtén tu clave API registrándote en https://www.pinecone.io/
# En producción, esta clave debe estar en variables de entorno, no en el código
api_key="provide your apikey"

In [ ]:
# Instalar las dependencias necesarias para trabajar con Pinecone y LangChain
# -qU: opciones para instalar de forma silenciosa (quiet) y actualizar (upgrade)
# langchain: framework principal para construir aplicaciones con LLMs
# langchain-pinecone: integración de LangChain con Pinecone para almacenamiento vectorial
# langchain-openai: integración para usar modelos y embeddings de OpenAI
!pip install -qU langchain langchain-pinecone langchain-openai

In [ ]:
# Importar la clase Pinecone para interactuar con la base de datos vectorial de Pinecone
from pinecone import Pinecone

# Crear una instancia del cliente de Pinecone usando la clave API
# Este objeto 'pc' se usará para crear y gestionar índices en Pinecone
pc=Pinecone(api_key=api_key)

In [ ]:
# Importar la clase OpenAIEmbeddings para crear embeddings usando modelos de OpenAI
from langchain_openai import OpenAIEmbeddings

# Crear una instancia del modelo de embeddings de OpenAI
# model: "text-embedding-3-small" es un modelo eficiente y económico de embeddings
# dimensions: 1024 especifica el tamaño del vector (debe coincidir con la dimensión del índice de Pinecone)
# api_key: clave de API de OpenAI para autenticación (IMPORTANTE: usar variables de entorno en producción)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",dimensions=1024,api_key="openai_api_key")

In [ ]:
# Importar ServerlessSpec para configurar el índice en modo serverless (sin servidor)
from pinecone import ServerlessSpec

# Nombre del índice que se creará en Pinecone (puedes cambiarlo si lo deseas)
index_name = "rag"  # cambiar si se desea

# Verificar si el índice ya existe en Pinecone
# Si no existe, crear un nuevo índice con las especificaciones indicadas
if not pc.has_index(index_name):
    # Crear el índice en Pinecone
    pc.create_index(
        name=index_name,  # Nombre del índice
        dimension=1024,  # Dimensión de los vectores (debe coincidir con los embeddings)
        metric="cosine",  # Métrica de similitud: "cosine" para similitud del coseno
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),  # Configuración serverless en AWS región us-east-1
    )

# Obtener una referencia al índice (ya sea recién creado o existente)
# Este objeto 'index' se usará para todas las operaciones en el índice
index = pc.Index(index_name)

In [ ]:
# Mostrar el objeto índice para verificar que se conectó correctamente
index

In [ ]:
# Importar PineconeVectorStore para usar Pinecone como almacén de vectores con LangChain
from langchain_pinecone import PineconeVectorStore

# Crear una instancia del vector store que conecta LangChain con Pinecone
# index: el índice de Pinecone donde se almacenarán los vectores
# embedding: el modelo de embeddings de OpenAI que convertirá textos en vectores
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
# Mostrar el objeto vector_store para verificar la configuración
vector_store

In [ ]:
# Importar la clase Document para crear objetos de documento con contenido y metadatos
from langchain_core.documents import Document

# Crear el primer documento: Tweet sobre desayuno
document_1 = Document(
    page_content="Desayuné panqueques con chispas de chocolate y huevos revueltos esta mañana.",
    metadata={"source": "tweet"},
)

# Crear el segundo documento: Noticia sobre el clima
document_2 = Document(
    page_content="El pronóstico del tiempo para mañana es nublado y cubierto, con una máxima de 62 grados.",
    metadata={"source": "news"},
)

# Crear el tercer documento: Tweet sobre un proyecto con LangChain
document_3 = Document(
    page_content="Construyendo un nuevo proyecto emocionante con LangChain - ¡ven a verlo!",
    metadata={"source": "tweet"},
)

# Crear el cuarto documento: Noticia sobre un robo bancario
document_4 = Document(
    page_content="Los ladrones asaltaron el banco de la ciudad y robaron 1 millón de dólares en efectivo.",
    metadata={"source": "news"},
)

# Crear el quinto documento: Tweet sobre una película
document_5 = Document(
    page_content="¡Guau! Esa fue una película increíble. No puedo esperar para verla de nuevo.",
    metadata={"source": "tweet"},
)

# Crear el sexto documento: Artículo web sobre el iPhone
document_6 = Document(
    page_content="¿Vale la pena el precio del nuevo iPhone? Lee esta reseña para descubrirlo.",
    metadata={"source": "website"},
)

# Crear el séptimo documento: Artículo web sobre futbolistas
document_7 = Document(
    page_content="Los 10 mejores jugadores de fútbol del mundo en este momento.",
    metadata={"source": "website"},
)

# Crear el octavo documento: Tweet sobre LangGraph
document_8 = Document(
    page_content="¡LangGraph es el mejor framework para construir aplicaciones con estado y agentes!",
    metadata={"source": "tweet"},
)

# Crear el noveno documento: Noticia sobre la bolsa de valores
document_9 = Document(
    page_content="El mercado de valores cayó 500 puntos hoy debido a temores de una recesión.",
    metadata={"source": "news"},
)

# Crear el décimo documento: Tweet con sentimiento negativo
document_10 = Document(
    page_content="Tengo el mal presentimiento de que voy a ser eliminado :(",
    metadata={"source": "tweet"},
)

# Crear una lista con todos los documentos para facilitar su procesamiento
documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [ ]:
# Agregar todos los documentos al vector store de Pinecone
# Este método convierte cada documento en un vector usando embeddings de OpenAI
# y los almacena en el índice de Pinecone en la nube
# Retorna una lista de IDs únicos (UUIDs) generados para cada documento almacenado
vector_store.add_documents(documents=documents)

In [ ]:
### Consulta Directa

# Realizar una búsqueda de similaridad con filtros en el vector store de Pinecone
# La consulta busca documentos sobre LangChain y abstracciones para trabajar con LLMs
# k=2: limita los resultados a los 2 documentos más similares
# filter={"source": "tweet"}: solo busca en documentos cuya fuente sea "tweet"
results = vector_store.similarity_search(
    "LangChain proporciona abstracciones para facilitar el trabajo con LLMs",
    k=2,
    filter={"source": "tweet"},
)
# Iterar sobre los resultados e imprimir el contenido y metadatos de cada documento
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [ ]:
# Realizar una búsqueda de similaridad con puntuación de similitud
# similarity_search_with_score retorna tanto los documentos como su puntuación de similitud
# La consulta pregunta sobre el clima de mañana
# k=1: retorna solo el documento más similar
# filter={"source": "news"}: filtra solo documentos de noticias
# La puntuación indica qué tan similar es el documento a la consulta
results = vector_store.similarity_search_with_score(
    "¿Hará calor mañana?", k=1, filter={"source": "news"}
)
# Mostrar cada resultado con su puntuación de similitud formateada
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

In [ ]:
### Retriever (Recuperador)

# Convertir el vector store en un retriever con búsqueda por umbral de similitud
# search_type="similarity_score_threshold": solo retorna documentos con similitud >= al umbral
# search_kwargs: parámetros de búsqueda
#   - k=1: retorna máximo 1 documento
#   - score_threshold=0.4: solo retorna documentos con puntuación de similitud >= 0.4
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.4},
)
# Invocar el retriever con una consulta sobre crimen bancario
# filter={"source": "news"}: filtra solo documentos de noticias
# Retorna los documentos que cumplen con el umbral de similitud
retriever.invoke("Robar del banco es un crimen", filter={"source": "news"})